In [ ]:
import torch
from torch.utils.data import Dataset
import os 
from path import Path

In [4]:
import open3d as o3d
import numpy as np
filename_base = "slam"
for i in range(250, 360, 9):
    filename = "/home/akshay/Deep_Closest_Point/" + filename_base + "/sample_input_" + str(i) + ".npy"
    np_array = np.load(filename)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_array)
    o3d.visualization.draw_geometries([pcd])

    filename = "/home/akshay/Deep_Closest_Point/" + filename_base + "/sample_output_" + str(i) + ".npy"
    np_array = np.load(filename)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_array)
    o3d.visualization.draw_geometries([pcd])


FileNotFoundError: [Errno 2] No such file or directory: '/home/akshay/Deep_Closest_Point/slam/sample_input_358.npy'

In [ ]:
#### slam
